In [1]:
import gurobipy as gb
from gurobipy import GRB, quicksum

In [2]:
def ASPBC(J, M, d, e, t, b):
	R = range(1, len(J) + 1)
	aspbc = gb.Model("ASP-BC")
	
	x = aspbc.addVars([(j,m) for j in J for m in M], vtype=GRB.BINARY) 
	q = aspbc.addVars([(r,m) for r in R for m in M], vtype=GRB.BINARY) 
	y = aspbc.addVars([(j,r,m) for j in J for r in R for m in M], vtype=GRB.BINARY)
	
	Cmax = aspbc.addVar(vtype=GRB.CONTINUOUS, lb=0.0)

	aspbc.setObjective(Cmax, GRB.MINIMIZE)

	for m in M:
		aspbc.addConstr(
			Cmax >= quicksum(d[j] * x[j,m] for j in J) + quicksum(t * q[r,m] for r in R if r != 1), "(1)")
	
	for j in J:
		aspbc.addConstr(quicksum(x[j,m] for m in M) == 1, "(2)")
	
	for j in J:
		for m in M:
			aspbc.addConstr(quicksum(y[j,r,m] for r in R) == x[j, m], "(3)")

	for j in J:
		for r in R:
			for m in M:
				aspbc.addConstr(2 * y[j,r,m] <= x[j,m] + q[r,m], "(4)")

	for r in R:
		for m in M:
			aspbc.addConstr(quicksum(e[j] * y[j,r,m] for j in J) <= b, "(5)")
	
	for r in R:
		if r != 1:
			for m in M:
				aspbc.addConstr(q[r,m] <= q[r-1,m], "(6)")

	for m in M:
		aspbc.addConstr(q[1,m] == 1, "(7)")

	aspbc.optimize()

	return aspbc, x, q, y, Cmax
	

In [ ]:
# aggiungere funzione controllo parametri (es. se batteria < durata job non è fattibile)

In [ ]:
if __name__ == "__main__":
    J = [1, 2, 3, 4]                # job
    M = ["A", "B"]                  # veicoli AGV
    R = J                           # lista slot di ricarica
    d = {1: 1, 2: 2, 3: 3, 4: 4}    # durata dei job
    e = {1: 6, 2: 3, 3: 1, 4: 6}    # energia necessaria per ogni job
    t = 2                           # tempo di ricarica
    b = 11                          # batteria di ogni AVG

    model, x, q, y, Cmax = ASPBC(J, M, d, e, t, b)

    print("****************************************************************")
    if model.status == 2:  # GRB.OPTIMAL
        print(f"Cmax ottimo = {Cmax.X:.2f}")
        print("Assegnazioni di job agli AGV:")
        for m in M:
            for j in J:
                if x[j,m].X > 0.5:
                    print(f"  job {j} -> AGV {m}")
        print("Segmenti di ricarica usati dagli AGV:")
        for m in M:
            for r in R:
                if q[r,m].X > 0.5:
                    print(f"  AGV {m} usa segmento di ricarica r={r}")
        print("Run dopo le ricariche:")
        for m in M:
            for j in J:
                for r in R:
                    if y[j,r,m].X > 0.5:
                        print(f"  job {j} su AGV {m} dopo la ricarica {r}")
    else:
        print(f"Errore: {model.status}")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G84)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 62 rows, 49 columns and 206 nonzeros
Model fingerprint: 0xb23b9f4e
Variable types: 1 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 16.0000000
Presolve removed 46 rows and 23 columns
Presolve time: 0.00s
Presolved: 16 rows, 26 columns, 60 nonzeros
Variable types: 0 continuous, 26 integer (25 binary)
Found heuristic solution: objective 11.0000000

Root relaxation: objective 5.000000e+00, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.00000  